In [ ]:
import numpy as np
import pandas as pd
import os
import os.path as opath
import ast
import json
from pandas.io.json import json_normalize
import datetime
import pickle

<div class="span5 alert alert-info">
<h3>Note book details</h3>

<p> This notebook is part 1 of 2 for data wrangling for <b>kickstarter success prediction</b> project.</p>

<p> Notes.</p>

<ol>
<li>  The file read the multiple csv files as input for each month </li> 
<li>  The file needs to be run manually for each month as files for following reason 
    <ul>
     <li> The files are available at end of months</li> 
    <li>  Processing each month is time consuming and resource extensive process on local computer </li> 
    </ul>
</li> 
<li>  The files are pickeld and csv to use further in project . This make easier to use data in later stages</li> 
    
</ol>
</div>

In [ ]:
# reading all files in monthly folder
df_month =pd.DataFrame()
folderpath='data/Kickstarter_2018-01-12T10_20_09_196Z'
picklefilename='df_Jan18_f1.sa'
csvfilename='df_Jan18_f1.csv'

for root,_,files in os.walk(folderpath):
    for f in files:
        fullpath=opath.join(root,f)
        df_file=pd.read_csv(fullpath)
        df_month= df_month.append(df_file,ignore_index=True)
df_month.set_index('id').sort_index()

## Wrangling and extracting values from sub values in column. The values are as Json object which needed to be parsed.

### This step is core of extracting data . Hence  This step runs for few hours on local machine hence some values printed in steps to confirm if process is still running 

In [ ]:
# getting coulmns needed to get the values
f=f[['id','name',
           'category',
           'goal','pledged','state','staff_pick','spotlight','backers_count','country','currency','usd_pledged',
           'deadline','state_changed_at','created_at','launched_at',
           'creator',# name of creator
           'location',# city 
           'urls',#urlto browse
           'disable_communication','slug']]#.head(10)
df_userinfo = pd.DataFrame()

# iterating over all rows to extract data
for index, row in f.iterrows():
   # Json normalizing the row to get namd and unique id for the project 
    uinfo=(json_normalize(json.loads(row.iloc[16]),record_path ='urls', meta=['name','id']))#, meta=['name','is_registered', 'id']))
    uinfo['projectid']=row.id#f.iloc[index].id
    
    # getting category value id 
    uinfo['Catgegory_val']=pd.read_json((row.iloc[2])).iloc[0][2]
    
    # getting deadline value
    uinfo['deadline_val']=( datetime.datetime.fromtimestamp(
        int(row['deadline'] )
    ).strftime('%Y-%m-%d %H:%M:%S'))
    uinfo['deadline_val']=( datetime.datetime.fromtimestamp(
        int(row['deadline'] )
    ).strftime('%Y-%m-%d %H:%M:%S'))
    
    # getting state change value
    uinfo['state_changed_at_val']=( datetime.datetime.fromtimestamp(
        int(row['state_changed_at'] )
    ).strftime('%Y-%m-%d %H:%M:%S'))
    
    # getting created at value
    uinfo['created_at_val']=( datetime.datetime.fromtimestamp(
        int(row['created_at'] )
    ).strftime('%Y-%m-%d %H:%M:%S'))
    
    # getting launched at value
    uinfo['launched_at_val']=( datetime.datetime.fromtimestamp(
        int(row['launched_at'] )
    ).strftime('%Y-%m-%d %H:%M:%S'))
    
    # print to check process running
    print(row.id)
    print(index)
    # getting location related value ( city, state type)
    if(str(row.iloc[17])!='nan'):
        linfo=pd.read_json(str(row.iloc[17]),encoding='UTF-8')
        uinfo['location_cityname']=linfo.iloc[0][4]
        uinfo['location_state']=linfo.iloc[0][7]
        uinfo['location_type']=linfo.iloc[0][8]
    else:
        uinfo['location_cityname']=np.NaN
        uinfo['location_state']=np.NaN
        uinfo['location_type']=np.NaN
    
    # getting project url for kickstarter with intention of web scrappinn information if required
    uinfo['Project_url']=(pd.read_json((row.iloc[18]))).iloc[0][0]
    
    # print to check end of processing of one row
    df_userinfo=df_userinfo.append(uinfo.iloc[1:,1:],ignore_index=True)
    print('**************************')

# Renaming coulms to more meaningful names
df_userinfo.rename(columns={'name': 'username', 'id': 'userid','projectid':'id'}, inplace=True)
f1=f.merge(df_userinfo,on='id')

#f1#.info()

### Pickle and save the single monthly file for further processing 

In [ ]:
# get the pickel file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)
f1.info()

In [ ]:
# create pickle file for further use 
pickle.dump(f1,open(picklefilename,'wb'))

In [ ]:
# create csv file for further use 
f1.to_csv(csvfilename,na_rep='NaN',encoding='UTF-8')

In [ ]:
# check pickel file
upickle_df = pickle.load(open(picklefilename,"rb"))

In [ ]:
upickle_df.info()

In [ ]:
upickle_df.head()